In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import json
import datetime

## Importing data

In [84]:
matches = pd.read_csv("matches.csv")
matches.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [4]:
attributes = pd.read_csv("attributes.csv")
attributes.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [26]:
batsmen = open('batsmen.json',) 
batsmen_data = json.load(batsmen)

In [7]:
bowlers = open('bowlers.json',) 
bowlers_data = json.load(bowlers)

In [12]:
invtmap = open('invtmap.json',) 
invtmap_data = json.load(invtmap)

In [9]:
scorecard = open('scorecard.json',) 
scorecard_data = json.load(scorecard)

In [23]:
region = open('region.json',) 
region_data = json.load(region)

In [17]:
tmap = open('tmap.json',) 
tmap_data = json.load(tmap)

## Making the Database

In [15]:
countries = list(set(matches['Team_1'].tolist()) | set(matches['Team_2'].tolist()))
countries

['AUS',
 'IND',
 'PAK',
 'WIN',
 'BAN',
 'ZIM',
 'ENG',
 'AFG',
 'IRE',
 'SAF',
 'KEN',
 'NED',
 'SRL',
 'NZL',
 'SCO']

In [27]:
ground = list(set(matches['GroundCode'].tolist()) )
#ground

In [82]:
#Input
team_1 = 'IND'
team_2 = 'ZIM' 
venue = 26
date = '2015-07-12'

match_1 = matches[matches['Date']==date]
match = match_1[match_1['Team_1']==team_1]
match_code = match['MatchCode'].tolist()[0]
score_card = scorecard_data[str(match_code)]
team_1_players = [score_card['BATTING1'][i][0] for i in range(10)]
team_2_players = [score_card['BATTING2'][i][0] for i in range(10)]
#name
team_1_players_name = [attributes[attributes['Code']==team_1_players[i]].iloc[0]['Full Name'] for i in range(10)]
team_2_players_name = [attributes[attributes['Code']==team_2_players[i]].iloc[0]['Full Name'] for i in range(10)]
#Batting style
team_1_players_bats = [attributes[attributes['Code']==team_1_players[i]].iloc[0]['Bats'] for i in range(10)]
team_2_players_bats = [attributes[attributes['Code']==team_2_players[i]].iloc[0]['Bats'] for i in range(10)]
#Bowling style
team_1_players_bowls = [attributes[attributes['Code']==team_1_players[i]].iloc[0]['Bowls'] for i in range(10)]
team_2_players_bowls = [attributes[attributes['Code']==team_2_players[i]].iloc[0]['Bowls'] for i in range(10)]
#Form for each player


['Right Arm Medium',
 'Right Arm Off Break',
 'Right Arm Off Break',
 'Right Arm Leg Break',
 'Right Arm Medium',
 'Right Arm Medium',
 'Right Arm Off Break',
 'Right Arm Off Break',
 'Left Arm Orthodox',
 'Right Arm Medium']

In [152]:
years_to_take_in = 5 #number of years to go back for dataset
matches_needed_for_dataset = matches[matches['Team_1']==team_1]
matches_needed_for_dataset = matches_needed_for_dataset[matches_needed_for_dataset['Team_2']==team_2]
matches_needed_for_dataset['Date']= pd.to_datetime(matches_needed_for_dataset['Date'])
date_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
date_obj_min = datetime.datetime.strptime(str(date_obj.year-years_to_take_in)+'-01'+'-01', '%Y-%m-%d')
matches_needed_for_dataset = matches_needed_for_dataset[matches_needed_for_dataset['Date']>=date_obj_min]
Dataset = matches_needed_for_dataset.reset_index()
MatchCodes = Dataset['MatchCode'].tolist()
Dataset = Dataset.drop(columns=['index'])

In [153]:
Dataset['Toss'] = ''
Dataset['Fours_team1'] = ''
Dataset['Fours_team2'] = ''
Dataset['Sixes_team1'] = ''
Dataset['Sixes_team2'] = ''
Dataset['Strike_rate_team1'] = ''
Dataset['Strike_rate_team2'] = ''
Dataset['Maidens_team1'] = ''
Dataset['Maidens_team2'] = ''
Dataset['Wickets_team1'] = ''
Dataset['Wickets_team2'] = ''
Dataset['Economy_rate_team1'] = ''
Dataset['Economy_rate_team2'] = ''

In [154]:
for i in range(len(MatchCodes)):
    matchdata = scorecard_data[str(MatchCodes[i])]
    #Dataset = Dataset.drop(columns=['GroundCode','Team_1','Team_2'])
    if (matchdata['TOSS']==matchdata['ORDER'][0]):
        toss = team_1
    else:
        toss = team_2
    #Batting 1
    batting1 = matchdata['BATTING1']
    Runs_team1 = []
    Balls_Played_team1 = []
    Fours_team1 = []
    Sixes_team1 = []
    Strike_rate_team1 = []
    for val in range(10):
        Runs_team1.append(batting1[val][2])
        Balls_Played_team1.append(batting1[val][3])
        Fours_team1.append(batting1[val][4])
        Sixes_team1.append(batting1[val][5])
        Strike_rate_team1.append(batting1[val][6])
        
    #Batting 2
    batting2 = matchdata['BATTING2']
    Runs_team2 = []
    Balls_Played_team2 = []
    Fours_team2 = []
    Sixes_team2 = []
    Strike_rate_team2 = []
    for val in range(10):
        Runs_team2.append(batting2[val][2])
        Balls_Played_team2.append(batting2[val][3])
        Fours_team2.append(batting2[val][4])
        Sixes_team2.append(batting2[val][5])
        Strike_rate_team2.append(batting2[val][6])

    #Bowling 1
    bowling1 = matchdata['BOWLING1']
    Overs_team1 = []
    Maidens_team1 = []
    Runs_given_team1 = []
    Wickets_team1 = []
    Economy_rate_team1 = []
    for val in range(10):
        if val<len(bowling1):
            Overs_team1.append(bowling1[val][2])
            Maidens_team1.append(bowling1[val][3])
            Runs_given_team1.append(bowling1[val][4])
            Wickets_team1.append(bowling1[val][5])
            Economy_rate_team1.append(bowling1[val][6])
        else:
            Overs_team1.append(-1)
            Maidens_team1.append(-1)
            Runs_given_team1.append(-1)
            Wickets_team1.append(-1)
            Economy_rate_team1.append(-1)

    #Bowling 2
    bowling2 = matchdata['BOWLING2']
    Overs_team2 = []
    Maidens_team2 = []
    Runs_given_team2 = []
    Wickets_team2 = []
    Economy_rate_team2 = []
    for val in range(10):
        if val<len(bowling2):
            Overs_team2.append(bowling2[val][1])
            Maidens_team2.append(bowling2[val][2])
            Runs_given_team2.append(bowling2[val][3])
            Wickets_team2.append(bowling2[val][4])
            Economy_rate_team2.append(bowling2[val][5])
        else:
            Overs_team2.append(-1)
            Maidens_team2.append(-1)
            Runs_given_team2.append(-1)
            Wickets_team2.append(-1)
            Economy_rate_team2.append(-1)
    
    #insert into dataset
    Dataset['Toss'].iloc[i] = toss
    Dataset['Fours_team1'].iloc[i] = Fours_team1
    Dataset['Fours_team2'].iloc[i] = Fours_team2
    Dataset['Sixes_team1'].iloc[i] = Sixes_team1
    Dataset['Sixes_team2'].iloc[i] = Sixes_team2
    Dataset['Strike_rate_team1'].iloc[i] = Strike_rate_team1
    Dataset['Strike_rate_team2'].iloc[i] = Strike_rate_team2
    Dataset['Maidens_team1'].iloc[i] = Maidens_team1
    Dataset['Maidens_team2'].iloc[i] = Maidens_team2
    Dataset['Wickets_team1'].iloc[i] = Wickets_team1
    Dataset['Wickets_team2'].iloc[i] = Wickets_team2
    Dataset['Economy_rate_team1'].iloc[i] = Economy_rate_team1
    Dataset['Economy_rate_team2'].iloc[i] = Economy_rate_team2

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [155]:
Dataset

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,Toss,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2
0,2010-05-28,IND,ZIM,Away,3111,73,IND,"[2.0, 1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1]","[1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1]","[0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1, -1]","[56.41, 45.83, 0.0, 95.8, 69.81, 100.0, 220.0,...","[45.83, 0.0, 95.8, 69.81, 100.0, 220.0, -1, -1...","[63.0, 36.0, 53.0, 46.0, 40.0, 40.0, 5.0, -1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.3, 18.0, 5.3, 4.6, 4.0, 5.71, 5.0, -1, -1, -1]","[0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1]","[0.2, 0, 0, 0.1, 0, 0, 0, -1, -1, -1]","[6.68, 6.38, 6.0, 4.7, 5.1, 7.0, 5.0, -1, -1, -1]"
1,2010-06-03,IND,ZIM,Away,3116,26,ZIM,"[4.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, ...","[70.21, 37.5, 62.07, 52.0, 60.0, 70.83, 60.0, ...","[37.5, 62.07, 52.0, 60.0, 70.83, 60.0, 0.0, 94...","[22.0, 37.0, 45.0, 41.0, 42.0, -1, -1, -1, -1,...","[0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[2.2, 3.7, 4.5, 4.1, 4.2, -1, -1, -1, -1, -1]","[0.0, 1.0, 0.0, 0.0, 2.0, -1, -1, -1, -1, -1]","[0.1, 0.1, 0.3, 0.1, 0.1, -1, -1, -1, -1, -1]","[6.48, 3.5, 5.0, 6.22, 3.86, -1, -1, -1, -1, -1]"
2,2013-07-24,IND,ZIM,Away,3537,26,ZIM,"[3.0, 6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1]","[6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1, -1]","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1]","[2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1, -1]","[47.22, 73.21, 51.72, 91.67, 80.0, 28.57, 126....","[73.21, 51.72, 91.67, 80.0, 28.57, 126.47, 114...","[57.0, 45.0, 39.0, 33.0, 43.0, 2.0, -1, -1, -1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.33, 4.5, 3.9, 3.3, 4.3, 2.0, -1, -1, -1, -1]","[1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.1, 0.1, 0.1, 0, 0.3, 0.1, -1, -1, -1, -1]","[5.0, 3.33, 6.0, 3.4, 8.3, 6.0, 6.0, -1, -1, -1]"
3,2013-07-26,IND,ZIM,Away,3539,26,ZIM,"[0.0, 11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0,...","[11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0, 0.0,...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, -1]","[14.29, 91.34, 77.78, 20.0, 36.36, 93.24, 68.1...","[91.34, 77.78, 20.0, 36.36, 93.24, 68.18, 128....","[70.0, 27.0, 46.0, 30.0, 67.0, 15.0, 34.0, -1,...","[2.0, 0.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1]","[7.0, 3.38, 5.11, 5.0, 6.7, 3.75, 11.33, -1, -...","[0.0, 1.0, 4.0, 1.0, 2.0, 0.0, -1, -1, -1, -1]","[0.1, 0.2, 0.1, 0, 0.1, 0, 0.1, -1, -1, -1]","[5.44, 5.2, 4.1, 3.0, 4.6, 5.0, -1, -1, -1, -1]"
4,2013-07-28,IND,ZIM,Away,3541,26,ZIM,"[0.0, 0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, ...","[0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26...","[33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26.32, ...","[32.0, 25.0, 24.0, 7.0, 39.0, 4.0, 47.0, -1, -...","[0.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[4.57, 2.78, 3.43, 7.0, 3.9, 2.0, 4.7, -1, -1,...","[1.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[0.1, 0.2, 0.1, 0, 0.1, 0, 0.4, -1, -1, -1]","[6.63, 5.14, 4.53, 4.1, 7.0, -1, -1, -1, -1, -1]"
5,2013-08-01,IND,ZIM,Away,3544,73,ZIM,"[3.0, 1.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[53.33, 41.18, 66.67, 0.0, 0.0, 45.45, 75.76, ...","[41.18, 66.67, 0.0, 0.0, 45.45, 75.76, 8.33, 1...","[26.0, 34.0, 27.0, 28.0, 25.0, -1, -1, -1, -1,...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.6, 4.25, 3.86, 3.11, 2.88, -1, -1, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -

In [156]:
Dataset = Dataset.drop(columns=['Date','Team_1','Team_2','MatchCode','GroundCode'])
Dataset

,Venue,Toss,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2
0,Away,IND,"[2.0, 1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1]","[1.0, 0.0, 6.0, 2.0, 5.0, 1.0, -1, -1, -1, -1]","[0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1]","[0.0, 0.0, 4.0, 0.0, 0.0, 1.0, -1, -1, -1, -1]","[56.41, 45.83, 0.0, 95.8, 69.81, 100.0, 220.0,...","[45.83, 0.0, 95.8, 69.81, 100.0, 220.0, -1, -1...","[63.0, 36.0, 53.0, 46.0, 40.0, 40.0, 5.0, -1, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.3, 18.0, 5.3, 4.6, 4.0, 5.71, 5.0, -1, -1, -1]","[0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1]","[0.2, 0, 0, 0.1, 0, 0, 0, -1, -1, -1]","[6.68, 6.38, 6.0, 4.7, 5.1, 7.0, 5.0, -1, -1, -1]"
1,Away,ZIM,"[4.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, ...","[70.21, 37.5, 62.07, 52.0, 60.0, 70.83, 60.0, ...","[37.5, 62.07, 52.0, 60.0, 70.83, 60.0, 0.0, 94...","[22.0, 37.0, 45.0, 41.0, 42.0, -1, -1, -1, -1,...","[0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[2.2, 3.7, 4.5, 4.1, 4.2, -1, -1, -1, -1, -1]","[0.0, 1.0, 0.0, 0.0, 2.0, -1, -1, -1, -1, -1]","[0.1, 0.1, 0.3, 0.1, 0.1, -1, -1, -1, -1, -1]","[6.48, 3.5, 5.0, 6.22, 3.86, -1, -1, -1, -1, -1]"
2,Away,ZIM,"[3.0, 6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1]","[6.0, 2.0, 2.0, 1.0, 0.0, 6.0, 0.0, 0.0, -1, -1]","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1]","[2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1, -1]","[47.22, 73.21, 51.72, 91.67, 80.0, 28.57, 126....","[73.21, 51.72, 91.67, 80.0, 28.57, 126.47, 114...","[57.0, 45.0, 39.0, 33.0, 43.0, 2.0, -1, -1, -1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[6.33, 4.5, 3.9, 3.3, 4.3, 2.0, -1, -1, -1, -1]","[1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.1, 0.1, 0.1, 0, 0.3, 0.1, -1, -1, -1, -1]","[5.0, 3.33, 6.0, 3.4, 8.3, 6.0, 6.0, -1, -1, -1]"
3,Away,ZIM,"[0.0, 11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0,...","[11.0, 2.0, 0.0, 0.0, 6.0, 1.0, 1.0, 2.0, 0.0,...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, -1]","[14.29, 91.34, 77.78, 20.0, 36.36, 93.24, 68.1...","[91.34, 77.78, 20.0, 36.36, 93.24, 68.18, 128....","[70.0, 27.0, 46.0, 30.0, 67.0, 15.0, 34.0, -1,...","[2.0, 0.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1]","[7.0, 3.38, 5.11, 5.0, 6.7, 3.75, 11.33, -1, -...","[0.0, 1.0, 4.0, 1.0, 2.0, 0.0, -1, -1, -1, -1]","[0.1, 0.2, 0.1, 0, 0.1, 0, 0.1, -1, -1, -1]","[5.44, 5.2, 4.1, 3.0, 4.6, 5.0, -1, -1, -1, -1]"
4,Away,ZIM,"[0.0, 0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, ...","[0.0, 5.0, 3.0, 2.0, 0.0, 0.0, 1.0, 2.0, 2.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0.0, 33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26...","[33.33, 71.7, 52.27, 84.91, 0.0, 37.5, 26.32, ...","[32.0, 25.0, 24.0, 7.0, 39.0, 4.0, 47.0, -1, -...","[0.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[4.57, 2.78, 3.43, 7.0, 3.9, 2.0, 4.7, -1, -1,...","[1.0, 1.0, 1.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[0.1, 0.2, 0.1, 0, 0.1, 0, 0.4, -1, -1, -1]","[6.63, 5.14, 4.53, 4.1, 7.0, -1, -1, -1, -1, -1]"
5,Away,ZIM,"[3.0, 1.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[53.33, 41.18, 66.67, 0.0, 0.0, 45.45, 75.76, ...","[41.18, 66.67, 0.0, 0.0, 45.45, 75.76, 8.33, 1...","[26.0, 34.0, 27.0, 28.0, 25.0, -1, -1, -1, -1,...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.6, 4.25, 3.86, 3.11, 2.88, -1, -1, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.2, 0.1, 0.1, 0.2, 0.3, -1, -1, -1, -1, -1]","[3.6, 3.5, 6.2, 4.67, 6.0, 5.25, 3.0, -1, -1, -1]"
6,Away,ZIM,"[1.0, 4.0, 0.0, 0.0, 6.0, 0.0, 3.0, 0.0, 0.0, ...","[4.0, 0.0, 0.0, 6.0, 0.0, 3.0,

## Feature Scaling

In [5]:
"""from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train)"""

'from sklearn.preprocessing import StandardScaler\nsc_X = StandardScaler()\nX_train = sc_X.fit_transform(X_train)\nX_test = sc_X.transform(X_test)\nsc_y = StandardScaler()\ny_train = sc_y.fit_transform(y_train)'

# Random Forest

In [4]:
# Test Train Split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/6, random_state = 0)


#fitting Random forest regression model to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train,y_train)


#Predicting a new result with Random forest regression 
y_pred = regressor.predict(X_test)

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result = result.append({'Model':'Random forest', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,33.155111,0.055753,0.382993,57.425032,0.775885
1,Random forest,32.161625,0.059738,0.384390,55.704211,0.767889
2,Random forest,38.447902,0.063565,0.472637,66.592012,0.714920


# Simple Linear Regression

In [7]:
#fitting linear model on the training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()    
regressor.fit(X_train,y_train)


#Predicting a new result with Simple Linear regression
y_pred = regressor.predict(X_test)

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result = result.append({'Model':'Simple Linear', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,18.861796,0.042359,0.042359,0.042359,0.887772
1,Simple Linear,17.843765,0.057774,0.057774,0.057774,0.836591


# Polynomial Regression

In [8]:
#Degree 2
#fitting polynomial regression 
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_test))

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result = result.append({'Model':'Polynomial (degree:2)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,18.861796,0.042359,0.042359,0.042359,0.887772
1,Simple Linear,17.843765,0.057774,0.057774,0.057774,0.836591
2,Polynomial (degree:2),14.267075,0.027728,0.027728,0.027728,0.940427


In [9]:
#Degree 3
#fitting polynomial regression 
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_test))

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result = result.append({'Model':'Polynomial (degree:3)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,18.861796,0.042359,0.042359,0.042359,0.887772
1,Simple Linear,17.843765,0.057774,0.057774,0.057774,0.836591
2,Polynomial (degree:2),14.267075,0.027728,0.027728,0.027728,0.940427
3,Polynomial (degree:3),13.904930,0.018888,0.018888,0.018888,0.954547


In [10]:
#Degree 4
#fitting polynomial regression 
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_test))

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result = result.append({'Model':'Polynomial (degree:4)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,18.861796,0.042359,0.042359,0.042359,0.887772
1,Simple Linear,17.843765,0.057774,0.057774,0.057774,0.836591
2,Polynomial (degree:2),14.267075,0.027728,0.027728,0.027728,0.940427
3,Polynomial (degree:3),13.904930,0.018888,0.018888,0.018888,0.954547
4,Polynomial (degree:4),16.365123,0.014614,0.014614,0.014614,0.956641


In [11]:
#Degree 5
#fitting polynomial regression 
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 5)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_test))

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result = result.append({'Model':'Polynomial (degree:5)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,18.861796,0.042359,0.042359,0.042359,0.887772
1,Simple Linear,17.843765,0.057774,0.057774,0.057774,0.836591
2,Polynomial (degree:2),14.267075,0.027728,0.027728,0.027728,0.940427
3,Polynomial (degree:3),13.904930,0.018888,0.018888,0.018888,0.954547
4,Polynomial (degree:4),16.365123,0.014614,0.014614,0.014614,0.956641
5,Polynomial (degree:5),42.097864,0.021710,0.021710,0.021710,0.837557


#### Model Testing

In [18]:
#Here, I used a Linear regression

#Fitting
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()    
regressor.fit(X,y)


#Predicting andd storing
y_pred = regressor.predict(X_new)
data = np.hstack((X_new,y_pred))
Prediction = pd.DataFrame(data = data,columns=['e(0)','rho(0)','K(0)','Q(0)','J(0)','L(0)','Ksym(0)','NS mass','R14','Lambda14'])
Prediction.head()

,e(0),rho(0),K(0),Q(0),J(0),L(0),Ksym(0),NS mass,R14,Lambda14
0,-16.689368,0.356295,229.486718,300.162569,32.450647,60.327011,-97.040015,0.941023,3.529158,-1299.406686
1,-13.908388,0.463953,230.344128,299.187913,32.416919,60.778809,-100.623579,0.331103,-1.503460,-2240.385749
2,-16.888786,0.253914,228.283242,299.926110,33.966100,60.546390,-99.052957,1.517631,8.203562,-428.683769
3,-17.313048,0.533067,232.019974,298.053390,33.229995,59.149342,-99.818871,-0.042387,-4.595037,-2809.848333
4,-17.509337,1.007855,229.939793,299.745767,32.874039,60.279793,-98.853342,-2.723195,-26.299675,-6868.880802


In [19]:
#Here, I used a Polynomial Regression with degree 2

#Fitting
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y)
#Predicting and storing
y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_new))
data = np.hstack((X_new,y_pred))
Prediction = pd.DataFrame(data = data,columns=['e(0)','rho(0)','K(0)','Q(0)','J(0)','L(0)','Ksym(0)','NS mass','R14','Lambda14'])
Prediction.head()

,e(0),rho(0),K(0),Q(0),J(0),L(0),Ksym(0),NS mass,R14,Lambda14
0,-16.689368,0.356295,229.486718,300.162569,32.450647,60.327011,-97.040015,0.269233,17.338007,2693.633260
1,-13.908388,0.463953,230.344128,299.187913,32.416919,60.778809,-100.623579,-0.929357,31.392716,7324.460856
2,-16.888786,0.253914,228.283242,299.926110,33.966100,60.546390,-99.052957,1.337024,11.458442,534.693514
3,-17.313048,0.533067,232.019974,298.053390,33.229995,59.149342,-99.818871,-2.457621,45.594234,11364.168788
4,-17.509337,1.007855,229.939793,299.745767,32.874039,60.279793,-98.853342,-14.394467,231.003054,65309.715942


In [20]:
#Here, I used a Polynomial Regression with degree 3

#Fitting
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(X)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y)
#Predicting and storing
y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_new))
data = np.hstack((X_new,y_pred))
Prediction = pd.DataFrame(data = data,columns=['e(0)','rho(0)','K(0)','Q(0)','J(0)','L(0)','Ksym(0)','NS mass','R14','Lambda14'])
Prediction.head()

,e(0),rho(0),K(0),Q(0),J(0),L(0),Ksym(0),NS mass,R14,Lambda14
0,-16.689368,0.356295,229.486718,300.162569,32.450647,60.327011,-97.040015,-12.758569,4.763907,-637.178970
1,-13.908388,0.463953,230.344128,299.187913,32.416919,60.778809,-100.623579,-79.143477,-105.249212,-21011.242356
2,-16.888786,0.253914,228.283242,299.926110,33.966100,60.546390,-99.052957,0.332789,9.856901,198.856696
3,-17.313048,0.533067,232.019974,298.053390,33.229995,59.149342,-99.818871,-92.636335,-24.515530,-9084.243976
4,-17.509337,1.007855,229.939793,299.745767,32.874039,60.279793,-98.853342,-1167.768530,-655.598708,-179102.621110


In [22]:
#Here, I used a Random Forest
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X,y)


#Predicting and Storing
y_pred = regressor.predict(X_new)
data = np.hstack((X_new,y_pred))
Prediction = pd.DataFrame(data = data,columns=['e(0)','rho(0)','K(0)','Q(0)','J(0)','L(0)','Ksym(0)','NS mass','R14','Lambda14'])
Prediction

,e(0),rho(0),K(0),Q(0),J(0),L(0),Ksym(0),NS mass,R14,Lambda14
0,-16.689368,0.356295,229.486718,300.162569,32.450647,60.327011,-97.040015,2.048871,12.299378,335.820744
1,-13.908388,0.463953,230.344128,299.187913,32.416919,60.778809,-100.623579,2.038524,12.297652,331.846573
2,-16.888786,0.253914,228.283242,299.926110,33.966100,60.546390,-99.052957,2.048708,12.301833,335.866040
3,-17.313048,0.533067,232.019974,298.053390,33.229995,59.149342,-99.818871,2.043302,12.231085,330.317366
4,-17.509337,1.007855,229.939793,299.745767,32.874039,60.279793,-98.853342,2.045901,12.295273,335.167230
5,-14.119812,0.392720,229.699934,298.630175,32.132218,57.676079,-100.449909,2.045102,12.173588,327.583831
6,-15.345226,0.156637,230.536196,298.585914,32.566179,58.904607,-100.871581,2.110165,12.618602,398.585902
7,-16.977605,1.995381,231.338948,299.115177,32.279380,59.942674,-100.077864,2.041364,12.280421,331.819700
8,-14.917212,1.461378,229.086234,299.357312,31.717561,59.051307,-99.450520,2.050868,12.210551,330.370951
9,-14.937975,2.551939,228.717289,299.803379,32.700192,58.145455,-99.290693,2.050900,12.165221,328.804663


#### Random Forest outperforms the other regression models tested

# Different Testings

In [30]:
#Training on C1 and C2 and testing on C3
NS_case1 = pd.read_excel("NS_EOS_Case1.xlsx")
NS_case2 = pd.read_excel("NS_EOS_Case2.xlsx")
NS_case3 = pd.read_excel("NS_EOS_Case3.xlsx")
NS = pd.concat([NS_case1,NS_case2],ignore_index=True)
X = NS.iloc[:, 0:7].values
y = NS.iloc[:, [7,9,11]].values
X_test = NS_case3.iloc[:, 0:7].values
y_test = NS_case3.iloc[:, [7,9,11]].values
result_new = pd.DataFrame(columns=['Model','RMS','RMS_NS_Mass','RMS_NS_Rad','RMS_Lam','R2'])

In [31]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X,y)


#Predicting
y_pred = regressor.predict(X_test)

#RMS value
rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

#R2 value
r2 = sklearn.metrics.r2_score(y_test,y_pred)

result_new = result_new.append({'Model':'Random forest', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                        'RMS_Lam':rms_lam, 'R2':r2}, ignore_index=True)
result_new

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2
0,Random forest,7.03024,0.026188,0.026188,0.026188,0.954838
